Diplomado IA
Nombre: Gustavo Alfredo Flores Pérez

Introducción

Recrear AlexNet de Scratch utilizando Pytorch an TensorFlow/Keras

La red debe tener 5 capas convolucionalesy 3 de max pool y 3 capas densas.
Se debe utilizar el dataset CIFAR -10


In [1]:
#Librerias que utilizaremos para nuestros modelos
import numpy as np
import matplotlib.pyplot as plt

# PyTorch Imports
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

# TensorFlow/Keras Imports
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import cifar10

# Configuracion del Dispositivo - para PyTorch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

Usando dispositivo: cuda


In [2]:
# --------------------------------------------------------------------------------
# PASO 2: Carga y Preprocesamiento del Dataset CIFAR-10
# --------------------------------------------------------------------------------

# Transformaciones para PyTorch
transform_pytorch = transforms.Compose([
    transforms.Resize(224),  # AlexNet espera imagenes de 224x224
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Dataset CIFAR-10 para PyTorch
trainset_pytorch = torchvision.datasets.CIFAR10(root='./data', train=True,
                                                download=True, transform=transform_pytorch)
trainloader_pytorch = torch.utils.data.DataLoader(trainset_pytorch, batch_size=100,
                                                  shuffle=True, num_workers=4)

testset_pytorch = torchvision.datasets.CIFAR10(root='./data', train=False,
                                               download=True, transform=transform_pytorch)
testloader_pytorch = torch.utils.data.DataLoader(testset_pytorch, batch_size=100,
                                                 shuffle=False, num_workers=4)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

print("Datasets CIFAR-10 para PyTorch cargados")

# Dataset CIFAR-10 para TensorFlow/Keras
(x_train_keras, y_train_keras), (x_test_keras, y_test_keras) = cifar10.load_data()

# Preprocesamiento para TensorFlow/Keras
def preprocess_keras(x):
    x = x.astype('float32') / 255.0
    # No normalizacion como en PyTorch para mantener la comparacion lo mas directa posible con el modelo scratch
    return x

x_train_keras = preprocess_keras(x_train_keras)
x_test_keras = preprocess_keras(x_test_keras)

y_train_keras = tf.keras.utils.to_categorical(y_train_keras, num_classes=10)
y_test_keras = tf.keras.utils.to_categorical(y_test_keras, num_classes=10)


print("Datasets CIFAR-10 para Keras cargados y preprocesados")

100%|██████████| 170M/170M [00:03<00:00, 43.5MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Files already downloaded and verified
Datasets CIFAR-10 para PyTorch cargados
170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Datasets CIFAR-10 para Keras cargados y preprocesados


In [12]:
# --------------------------------------------------------------------------------
# PASO 3: Definicion de Modelos AlexNet desde Cero
# --------------------------------------------------------------------------------

# --- 3.1 Modelo AlexNet desde Cero en PyTorch ---
class AlexNetScratch_PyTorch(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNetScratch_PyTorch, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1), # Menos params, kernel 3 en vez de 11
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), # kernel y stride modificados

            nn.Conv2d(64, 192, kernel_size=3, padding=1), # kernel 3 en vez de 5
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), # kernel y stride modificados

            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), # kernel y stride modificados
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6)) # Adaptado para entradas 224x224
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# --- 3.2 Modelo AlexNet desde Cero con TensorFlow/Keras ---
def AlexNetScratch_Keras(input_shape=(224, 224, 3), num_classes=10):
    model = Sequential([
        Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape), # Kernel y stride modificados
        MaxPooling2D((2, 2), strides=(2,2)), # Kernel y stride modificados

        Conv2D(192, (3, 3), activation='relu', padding='same'), # Kernel modificado
        MaxPooling2D((2, 2), strides=(2,2)), # Kernel y stride modificados

        Conv2D(384, (3, 3), activation='relu', padding='same'),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2), strides=(2,2)), # Kernel y stride modificados

        Flatten(),
        Dense(512, activation='relu'),  # <--- MODIFICADO a 512
        Dense(512, activation='relu'),  # <--- MODIFICADO a 512
        Dense(num_classes, activation='softmax') # softmax para clasificacion multiclase
    ])
    return model

# --- 3.3 Modelo Pre-entrenado AlexNet en PyTorch ---
pretrained_alexnet_pytorch = torchvision.models.alexnet(pretrained=True)
# Modifica la ultima capa para CIFAR-10 (10 clases)
num_features_pytorch = pretrained_alexnet_pytorch.classifier[6].in_features
pretrained_alexnet_pytorch.classifier[6] = nn.Linear(num_features_pytorch, 10)


print("Modelos AlexNet definidos (desde cero y pre-entrenado)")

Modelos AlexNet definidos (desde cero y pre-entrenado)


In [4]:
# --------------------------------------------------------------------------------
# PASO 4: Funciones de Entrenamiento y Evaluacion
# --------------------------------------------------------------------------------

# --- 4.1 Funciones para PyTorch ---
def train_pytorch_model(model, trainloader, criterion, optimizer, device, epochs=2):
    model.to(device) # Mueve el modelo a la GPU si está disponible
    model.train() # Modo entrenamiento
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad() # Limpia los gradientes anteriores
            outputs = model(inputs) # Pasa los datos a través del modelo
            loss = criterion(outputs, labels) # Calcula la pérdida
            loss.backward() # Backpropagation
            optimizer.step() # Actualiza los pesos
            running_loss += loss.item()
            if i % 2000 == 1999:    # Imprime cada 2000 mini-batches
                print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
                running_loss = 0.0
    print('Entrenamiento Finalizado (PyTorch)')

def evaluate_pytorch_model(model, testloader, device):
    model.to(device) # Asegura que el modelo esté en el mismo dispositivo que los datos
    model.eval() # Modo evaluacion
    correct = 0
    total = 0
    with torch.no_grad(): # Desactiva el cálculo de gradientes durante la evaluacion
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1) # Obtiene las predicciones de la clase
            total += labels.size(0) # Cuenta el numero total de muestras
            correct += (predicted == labels).sum().item() # Suma las predicciones correctas
    accuracy = 100 * correct / total
    print(f'Accuracy del modelo en PyTorch en las 10000 imagenes de prueba: {accuracy:.2f} %')
    return accuracy

# --- 4.2 Funciones para TensorFlow/Keras ---
def train_keras_model(model, x_train, y_train, x_val, y_val, epochs=2, batch_size=64):
    model.compile(optimizer=SGD(learning_rate=0.001, momentum=0.9), # Optimizador y learning rate como en PyTorch
                  loss='categorical_crossentropy', #  loss para clasificación multiclase
                  metrics=['accuracy']) # Metrica para evaluar

    history = model.fit(x_train, y_train,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(x_val, y_val), # Datos de validacion
                        verbose=1) # Muestra el progreso del entrenamiento

    print('Entrenamiento Finalizado (Keras)')
    return history

def evaluate_keras_model(model, x_test, y_test):
    _, accuracy = model.evaluate(x_test, y_test, verbose=0) # Evalua el modelo en datos de prueba
    print(f'Accuracy del modelo en Keras en las 10000 imagenes de prueba: {accuracy*100:.2f} %')
    return accuracy

print("Funciones de entrenamiento y evaluación definidas para PyTorch y Keras")

Funciones de entrenamiento y evaluación definidas para PyTorch y Keras


In [5]:
epochs = 5 # Numero de epochs para entrenamiento, puedes aumentarlo para mejores resultados

# --- 5.1 Entrenamiento de AlexNet desde Cero en PyTorch ---
alexnet_scratch_pytorch = AlexNetScratch_PyTorch()
criterion_pytorch = nn.CrossEntropyLoss()
optimizer_pytorch = optim.SGD(alexnet_scratch_pytorch.parameters(), lr=0.001, momentum=0.9)

print("\nEntrenando AlexNet desde cero en PyTorch...")
train_pytorch_model(alexnet_scratch_pytorch, trainloader_pytorch, criterion_pytorch, optimizer_pytorch, device, epochs)




Entrenando AlexNet desde cero en PyTorch...
Entrenamiento Finalizado (PyTorch)


In [6]:
# @title PASO 5.2: Evaluación de AlexNet desde Cero en PyTorch

# --- 5.2 Evaluación de AlexNet desde Cero en PyTorch ---
accuracy_scratch_pytorch = evaluate_pytorch_model(alexnet_scratch_pytorch, testloader_pytorch, device)
print(f"Accuracy AlexNet desde cero (PyTorch): {accuracy_scratch_pytorch:.2f} %")

Accuracy del modelo en PyTorch en las 10000 imagenes de prueba: 20.09 %
Accuracy AlexNet desde cero (PyTorch): 20.09 %


In [ ]:
# @title PASO 5.3: Creación de AlexNet desde Cero en TensorFlow/Keras

# --- 5.3 Creación de AlexNet desde Cero en TensorFlow/Keras ---
alexnet_scratch_keras = AlexNetScratch_Keras(input_shape=(224, 224, 3))

print("\nModelo AlexNet desde cero en Keras creado.")

In [ ]:
# @title PASO 5.4: Entrenamiento de AlexNet desde Cero en TensorFlow/Keras

# --- 5.4 Entrenamiento de AlexNet desde Cero en TensorFlow/Keras ---
print("\nEntrenando AlexNet desde cero en Keras...")
history_scratch_keras = train_keras_model(alexnet_scratch_keras, x_train_keras, y_train_keras, x_test_keras, y_test_keras, epochs) # Usamos test set como val set por simplicidad

In [ ]:
# @title PASO 5.5: Evaluación de AlexNet desde Cero en TensorFlow/Keras

# --- 5.5 Evaluación de AlexNet desde Cero en TensorFlow/Keras ---
accuracy_scratch_keras = evaluate_keras_model(alexnet_scratch_keras, x_test_keras, y_test_keras)
print(f"Accuracy AlexNet desde cero (Keras): {accuracy_scratch_keras*100:.2f} %") # Keras accuracy está en escala [0, 1]

In [ ]:
# @title PASO 5.6: Entrenamiento y Evaluación de AlexNet Pre-entrenado en PyTorch

# --- 5.6 Entrenamiento y Evaluación de AlexNet Pre-entrenado en PyTorch ---
pretrained_optimizer_pytorch = optim.SGD(pretrained_alexnet_pytorch.parameters(), lr=0.001, momentum=0.9) # Optimizador para modelo pre-entrenado

print("\nEntrenando AlexNet Pre-entrenado en PyTorch...")
train_pytorch_model(pretrained_alexnet_pytorch, trainloader_pytorch, criterion_pytorch, pretrained_optimizer_pytorch, device, epochs)
accuracy_pretrained_pytorch = evaluate_pytorch_model(pretrained_alexnet_pytorch, testloader_pytorch, device)
print(f"Accuracy AlexNet Pre-entrenado (PyTorch): {accuracy_pretrained_pytorch:.2f} %")

In [ ]:
# @title PASO 5.7: Resultados de Comparación

# --- 5.7: Resultados de Comparación ---
print("\n--- Resultados de Comparación ---")
print(f"Accuracy AlexNet desde cero (PyTorch): {accuracy_scratch_pytorch:.2f} %")
print(f"Accuracy AlexNet desde cero (Keras): {accuracy_scratch_keras*100:.2f} %") # Keras accuracy está en escala [0, 1]
print(f"Accuracy AlexNet Pre-entrenado (PyTorch): {accuracy_pretrained_pytorch:.2f} %")

print("\n¡Tarea completada!")

In [ ]:
# --------------------------------------------------------------------------------
# PASO 6: (Opcional) Visualización Adicional
# --------------------------------------------------------------------------------

# --- 6.1 Visualización de algunas imagenes de prueba y predicciones (PyTorch Scratch Model) ---
def imshow(img):
    img = img / 2 + 0.5     # Desnormalizar
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

dataiter = iter(testloader_pytorch)
images, labels = next(dataiter)

# Muestra imagenes
imshow(torchvision.utils.make_grid(images[:4]))
print('Etiquetas reales: ', ' '.join(classes[labels[:4]]))

# Prediccion con modelo scratch PyTorch
outputs_scratch_pytorch = alexnet_scratch_pytorch(images.to(device))
_, predicted_scratch_pytorch = torch.max(outputs_scratch_pytorch, 1)
print('Predicciones AlexNet Scratch PyTorch: ', ' '.join(classes[predicted_scratch_pytorch[:4]]))


# --- 6.2 (Opcional) Grafica de la historia de entrenamiento de Keras ---
if 'history_scratch_keras' in locals(): # Verifica si 'history_scratch_keras' existe
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history_scratch_keras.history['accuracy'])
    plt.plot(history_scratch_keras.history['val_accuracy'])
    plt.title('Accuracy del modelo Keras')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.subplot(1, 2, 2)
    plt.plot(history_scratch_keras.history['loss'])
    plt.plot(history_scratch_keras.history['val_loss'])
    plt.title('Pérdida del modelo Keras')
    plt.ylabel('Pérdida')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()